In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import MobileBertTokenizer, MobileBertForSequenceClassification, AdamW

In [3]:
# Load the dataset
data = pd.read_csv('/content/alexa_review_pruned.csv')  # Replace with your dataset path
reviews = data['Review'].values
labels = data['Liked'].values

# Split the dataset into training and testing sets
train_reviews, test_reviews, train_labels, test_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

UnicodeDecodeError: ignored

In [ ]:
# Load MobileBERT tokenizer and model
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
model = MobileBertForSequenceClassification.from_pretrained('google/mobilebert-uncased')

In [ ]:
# Tokenize and encode the training reviews
train_encodings = tokenizer(list(train_reviews), truncation=True, padding=True, max_length=512, return_tensors='pt')
train_labels = torch.tensor(train_labels)

In [ ]:
# Tokenize and encode the testing reviews
test_encodings = tokenizer(list(test_reviews), truncation=True, padding=True, max_length=512, return_tensors='pt')
test_labels = torch.tensor(test_labels)

# Set up the optimizer and the model
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
# Train the model
model.train()
optimizer.zero_grad()
outputs = model(input_ids=train_encodings['input_ids'], attention_mask=train_encodings['attention_mask'], labels=train_labels)
loss = outputs.loss
loss.backward()
optimizer.step()

In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(input_ids=test_encodings['input_ids'], attention_mask=test_encodings['attention_mask'])
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

In [ ]:
# Convert predictions and labels to numpy arrays
predictions = predictions.detach().numpy()
test_labels = test_labels.numpy()

In [ ]:
# Calculate evaluation metrics
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
